In [ ]:
from datetime import timedelta, datetime
import glob
import json
import os
import re

from collections import Counter

import numpy as np
import pandas as pd

import scipy.sparse as spr
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
result1 = pd.read_json('/content/drive/My Drive/Colab Notebooks/머신/sm_results.json', typ = 'frame',encoding='UTF-8')
result2 = pd.read_json('/content/drive/My Drive/Colab Notebooks/results1111.json', typ = 'frame',encoding='UTF-8')
result3 = pd.read_json('/content/drive/My Drive/Colab Notebooks/results33.json', typ = 'frame',encoding='UTF-8')

In [ ]:
result1.drop(['tags'], axis=1, inplace=True)
result2.drop(['tags'], axis=1, inplace=True)
result3.drop(['tags'], axis=1, inplace=True)

In [ ]:
data = pd.merge(result1, result2, on='id')
data = pd.merge(data, result3, on='id')

In [ ]:
data.head()

,id,songs_x,songs_y,songs
0,118598,"[207912, 623047, 703323, 422438, 322215, 63848...","[207912, 623047, 703323, 422438, 322215, 63848...","[207912, 623047, 422438, 703323, 322215, 78983..."
1,131447,"[188666, 409683, 697085, 394622, 64052, 120895...","[188666, 409683, 697085, 394622, 64052, 120895...","[188666, 409683, 697085, 394622, 64052, 120895..."
2,51464,"[154031, 411161, 230907, 596279, 35792, 396367...","[154031, 596279, 411161, 230907, 35792, 66347,...","[154031, 596279, 35792, 66347, 411161, 253755,..."
3,45144,"[144663, 367963, 224921, 174749, 629738, 19296...","[144663, 367963, 224921, 174749, 629738, 19296...","[144663, 367963, 629738, 224921, 174749, 19296..."
4,79929,"[70314, 211220, 445299, 412769, 522562, 106129...","[70314, 211220, 445299, 412769, 522562, 106129...","[70314, 211220, 445299, 412769, 522562, 106129..."


In [ ]:
data.shape

(23015, 4)

In [ ]:
rec = []

for i in tqdm_notebook(range(23015)):

  id = data.iloc[i,0]
  song1 = data.iloc[i,1]
  song2 = data.iloc[i,2]
  song3 = data.iloc[i,3]

  song_1 = pd.DataFrame(song1,columns=['song']).reset_index()
  song_1['index'] = 100-song_1['index']
  song_2 = pd.DataFrame(song2,columns=['song']).reset_index()
  song_2['index'] = 100-song_2['index']
  song_3 = pd.DataFrame(song3,columns=['song']).reset_index()
  song_3['index'] = 100-song_3['index']

  song_fin = pd.merge(song_1, song_2, on='song', how = 'outer').fillna(0)
  song_fin = pd.merge(song_fin, song_3, on='song', how='outer').fillna(0)

  song_fin['count'] = song_fin['index_x'] + song_fin['index_y'] + song_fin['index']
  song_fin[['song','count']].iloc[:100,:].song.to_list()

  rec.append(song_fin)

  rec[i] = rec[i][['song', 'count']][:100].sort_values('count', ascending=False).reset_index().drop(['index'], axis=1)
  rec[i] = rec[i]['song'].tolist()



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
rec[0]

[207912,
 623047,
 703323,
 422438,
 322215,
 638488,
 78983,
 157900,
 328908,
 690767,
 39436,
 516376,
 551157,
 140837,
 472374,
 66475,
 439161,
 607459,
 262430,
 413459,
 367826,
 569715,
 292859,
 411438,
 217622,
 209622,
 412246,
 448547,
 557956,
 447762,
 646988,
 654428,
 680970,
 187531,
 394031,
 585728,
 545816,
 219415,
 412305,
 562903,
 404536,
 455806,
 438857,
 693988,
 249378,
 491808,
 620311,
 467269,
 268669,
 359986,
 520034,
 381851,
 546713,
 125214,
 611201,
 107413,
 380092,
 370618,
 89503,
 365613,
 584108,
 421833,
 494552,
 586389,
 259770,
 389960,
 245272,
 76044,
 225652,
 212592,
 300170,
 78337,
 606063,
 416412,
 258135,
 375776,
 218898,
 307938,
 166439,
 653565,
 242188,
 226991,
 616144,
 303904,
 264753,
 671973,
 333970,
 637900,
 271335,
 182907,
 10690,
 12795,
 461170,
 164168,
 566502,
 425357,
 660172,
 328517,
 693580,
 673758]

In [ ]:
result1 = pd.read_json('/content/drive/My Drive/Colab Notebooks/머신/sm_results.json', typ = 'frame',encoding='UTF-8')
result1.head(2)

,id,songs,tags
0,118598,"[207912, 623047, 703323, 422438, 322215, 63848...","[OST, 디즈니, 애니메이션, 영화, 디즈니OST, 추억, 기분전환, 겨울왕국, ..."
1,131447,"[188666, 409683, 697085, 394622, 64052, 120895...","[OST, 디즈니, 애니메이션, 영화, 기분전환, 추억, 휴식, 힐링, 디즈니OST..."


In [ ]:
res = []
for i in result1.index :
  res.append({
      "id" : result1.iloc[i, 0],
      "songs" : rec[i],
      "tags":result1.iloc[i, 2]
  })

import io
import os

def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    with io.open(fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

write_json(res, '/content/drive/My Drive/Colab Notebooks/머신/rerankinng_results555.json')

## 이건 내 방법 

In [ ]:
result1_unnest = np.dstack(
(
np.repeat(result1.id.values, list(map(len, result1.songs))),
np.concatenate(result1.songs.values))
)

result1_map = pd.DataFrame(data=result1_unnest[0], columns = result1.columns)
result1_map.rename(columns = {'songs' : 'songs1'}, inplace = True)
result2_unnest = np.dstack(
(
np.repeat(result2.id.values, list(map(len, result2.songs))),
np.concatenate(result2.songs.values))
)

result2_map = pd.DataFrame(data=result2_unnest[0], columns = result2.columns)
result2_map.rename(columns = {'songs' : 'songs2'}, inplace = True)
result3_unnest = np.dstack(
(
np.repeat(result3.id.values, list(map(len, result3.songs))),
np.concatenate(result3.songs.values))
)

result3_map = pd.DataFrame(data=result3_unnest[0], columns = result3.columns)
result3_map.rename(columns = {'songs' : 'songs3'}, inplace = True)

In [ ]:
result1_map = result1_map.reset_index()
result1_map

,index,id,songs1
0,0,118598,207912
1,1,118598,623047
2,2,118598,703323
3,3,118598,422438
4,4,118598,322215
...,...,...,...
2301495,2301495,65189,417380
2301496,2301496,65189,194954
2301497,2301497,65189,227331
2301498,2301498,65189,5101


In [ ]:
result3_map = result3_map.reset_index()
result3_map

,index,id,songs3
0,0,118598,207912
1,1,118598,623047
2,2,118598,422438
3,3,118598,703323
4,4,118598,322215
...,...,...,...
2301495,2301495,65189,471988
2301496,2301496,65189,553437
2301497,2301497,65189,650764
2301498,2301498,65189,343974


In [ ]:
result11 = result1_map.loc[result1_map['id']==118598]
result11['count1'] = sorted(list(range(100)), reverse=True)
# result11.rename(columns={'songs1':'songs3'}, inplace=True)
result11

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,index,id,songs1,count1
0,0,118598,207912,99
1,1,118598,623047,98
2,2,118598,703323,97
3,3,118598,422438,96
4,4,118598,322215,95
...,...,...,...,...
95,95,118598,164168,4
96,96,118598,566502,3
97,97,118598,660172,2
98,98,118598,328517,1


In [ ]:
# d = result11.groupby(['songs1']).apply(lambda x : dict(zip(x.songs1, x.count1))).to_dict()

In [ ]:
result33 = result3_map.loc[result1_map['id']==118598]
result33['count3'] = sorted(list(range(100)), reverse=True)
result33.rename(columns={'songs3':'songs1'}, inplace=True)
result33.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,index,id,songs1,count3
0,0,118598,207912,99
1,1,118598,623047,98
2,2,118598,422438,97
3,3,118598,703323,96
4,4,118598,322215,95


In [ ]:
res = []

for i in tqdm_notebook(data['id']):
  result11 = result1_map.loc[result1_map['id']==i]
  result11['count1'] = sorted(list(range(100)), reverse=True)

  result33 = result3_map.loc[result1_map['id']==i]
  result33['count3'] = sorted(list(range(100)), reverse=True)
  result33.rename(columns={'songs3':'songs1'}, inplace=True)

  d1 = dict(zip(result11.songs1, result11.count1))
  d2 = dict(zip(result33.songs1, result33.count3))

  # 두 딕셔너리 결합
  from collections import Counter
  c1 = Counter(d1)
  c2 = Counter(d2)
  rr = c1 + c2

  df = pd.DataFrame.from_dict(rr, 'index').reset_index()
  df = df.rename(columns = {'index':'songs', 0:'count'})
  top100 = df.iloc[:100, :]

  res.append(top100)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

In [ ]:
result1 = pd.read_json('/content/drive/My Drive/Colab Notebooks/results1111.json', typ = 'frame',encoding='UTF-8')

In [ ]:
result1.head()

,id,songs,tags
0,118598,"[207912, 623047, 703323, 422438, 322215, 63848...","[OST, 디즈니, 애니메이션, 영화, 디즈니OST, 추억, 기분전환, 겨울왕국, ..."
1,131447,"[188666, 409683, 697085, 394622, 64052, 120895...","[OST, 디즈니, 애니메이션, 영화, 기분전환, 추억, 휴식, 힐링, 디즈니OST..."
2,51464,"[154031, 596279, 411161, 230907, 35792, 66347,...","[발라드, 이별, 슬픔, 추억, 회상, 밤, 새벽, 잔잔한, 설렘, 사랑]"
3,45144,"[144663, 367963, 224921, 174749, 629738, 19296...","[감성, 사랑, 이별, 발라드, 잔잔한, 새벽, 기분전환, 가을, 카페, 인디]"
4,79929,"[70314, 211220, 445299, 412769, 522562, 106129...","[CCM, 찬양, 예배, 은혜, 워십, 국내ccm, 교회, 찬송, 제이어스, 감사]"


In [ ]:
len(res)

23015

In [ ]:
res[0]

,songs,count
0,207912,198
1,623047,196
2,703323,193
3,422438,193
4,322215,190
...,...,...
95,164168,15
96,566502,13
97,660172,10
98,328517,8


In [ ]:
h = []
for i in tqdm_notebook(range(23015)):
  top100_list = res[i]['songs']
  h.append(top100_list)
  result1['songs'][i] = h[i]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
result1['songs'][0]

[207912,
 623047,
 703323,
 422438,
 322215,
 638488,
 78983,
 157900,
 39436,
 690767,
 328908,
 140837,
 516376,
 551157,
 439161,
 472374,
 569715,
 66475,
 607459,
 262430,
 367826,
 413459,
 292859,
 209622,
 217622,
 411438,
 557956,
 447762,
 412246,
 448547,
 646988,
 654428,
 394031,
 187531,
 585728,
 680970,
 545816,
 219415,
 412305,
 562903,
 404536,
 455806,
 438857,
 693988,
 491808,
 249378,
 620311,
 546713,
 467269,
 89503,
 380092,
 494552,
 611201,
 268669,
 359986,
 381851,
 125214,
 107413,
 520034,
 421833,
 370618,
 584108,
 365613,
 307938,
 259770,
 586389,
 389960,
 245272,
 212592,
 225652,
 300170,
 78337,
 226991,
 606063,
 258135,
 76044,
 375776,
 416412,
 182907,
 653565,
 218898,
 671973,
 166439,
 242188,
 10690,
 616144,
 264753,
 416417,
 352807,
 303904,
 333970,
 637900,
 271335,
 461170,
 12795,
 164168,
 566502,
 660172,
 328517,
 425357]

In [ ]:
res = []
for i in result1.index :
  res.append({
      "id" : result1.iloc[i, 0],
      "songs" : result1.iloc[i, 1],
      "tags":result1.iloc[i, 2]
  })

import io
import os

def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    with io.open(fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

write_json(res, '/content/drive/My Drive/Colab Notebooks/머신/rerankinng_results111.json')